In [3]:
import urllib.request, urllib.parse, urllib.error
import json
import time
import random
import requests
import sys, time
import pandas as pd 
from pandas.io.json import json_normalize
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Collecting data and formatting it. 
def getCourseURL():
    courserequested = input(str("Enter the course code: ")).upper()
    url_return = 'https://api.umd.io/v0/courses/'
    url = 'https://api.umd.io/v0/courses/list'
    uh = urllib.request.urlopen(url)
    print('retrieving', url)
    data = uh.read().decode()
    info = json.loads(data)
    #print(json.dumps(info, indent=4, separators=(". ", " = ")))
    courses_array = []
    for courselist in info:
        if courselist['course_id'].startswith(courserequested):
            courses_array.append(courselist['course_id'])
            #print(courselist['course_id'])
            #print(courses_array)
    myString = ",".join(courses_array)
    url_return=url_return+myString
    print(url_return)  
    return url_return

#print(getCourseURL(courserequested))

def getJSON(url):
    uh = urllib.request.urlopen(url)
    print('retrieving', url)
    data = uh.read().decode()
    info = json.loads(data)
    #making the file so you can read it in your script. 
    with open("coursedata.json", "w") as writeJSON:
        json.dump(json.dumps({"data": info}, indent=3), writeJSON, ensure_ascii=False)
    return (json.dumps({"data": info}, indent=4))
#getJSON(getCourseURL(courserequested))



In [5]:
# UMD.io Data Getting user input for course: 
df = pd.read_json(getCourseURL(), orient='columns')
#df[['course_id', 'description', 'name']].head(3)  len(df['sections'][1])
df['popularity'] = [len(c) for c in df['sections']] #this assigns a score based on the amount of sections, assumming past popularity merits more sections?!
df['popularity_avg'] = df['popularity']/df['popularity'].mean()
#a = df['grading_method'][1]
#print(len(a))
df.head(3)

Enter the course code: cmsc
retrieving https://api.umd.io/v0/courses/list
https://api.umd.io/v0/courses/CMSC122,CMSC131,CMSC132,CMSC132A,CMSC216,CMSC216H,CMSC250,CMSC250H,CMSC298A,CMSC320,CMSC330,CMSC351,CMSC389N,CMSC390,CMSC396H,CMSC411,CMSC412,CMSC414,CMSC417,CMSC420,CMSC421,CMSC422,CMSC423,CMSC424,CMSC425,CMSC426,CMSC433,CMSC434,CMSC435,CMSC436,CMSC451,CMSC452,CMSC456,CMSC457,CMSC460,CMSC466,CMSC470,CMSC474,CMSC475,CMSC498A,CMSC498L,CMSC498O,CMSC498U,CMSC499A,CMSC631,CMSC642,CMSC644,CMSC661,CMSC664,CMSC712,CMSC722,CMSC726,CMSC733,CMSC740,CMSC764,CMSC773,CMSC798,CMSC798F,CMSC799,CMSC818N,CMSC818W,CMSC818X,CMSC818Y,CMSC828B,CMSC828F,CMSC828U,CMSC828X,CMSC838J,CMSC858M,CMSC898,CMSC899


,core,course_id,credits,department,dept_id,description,gen_ed,grading_method,name,relationships,sections,semester,popularity,popularity_avg
0,[I],CMSC122,3,Computer Science,CMSC,Introduction to computer programming in the co...,[DSSP],[Regular],Introduction to Computer Programming via the Web,"{'prereqs': None, 'coreqs': None, 'restriction...","[CMSC122-0101, CMSC122-0201]",201901,2,0.830409
1,[],CMSC131,4,Computer Science,CMSC,Introduction to programming and computer scien...,[],[Regular],Object-Oriented Programming I,"{'prereqs': None, 'coreqs': 'MATH140; and perm...","[CMSC131-0101, CMSC131-0102, CMSC131-0103, CMS...",201901,20,8.304094
2,[],CMSC132,4,Computer Science,CMSC,Introduction to use of computers to solve prob...,[],[Regular],Object-Oriented Programming II,{'prereqs': 'Minimum grade of C- in CMSC131; o...,"[CMSC132-0101, CMSC132-0102, CMSC132-0103, CMS...",201901,23,9.549708


In [6]:
#Getting final version of what data i need and calulating the mean.  
#df[['course_id', 'description', 'name', 'rank']].head(15).to_csv('test.csv', sep='\t', encoding='utf-8')
df[['course_id', 'description', 'name', 'popularity', 'popularity_avg']].head(15)
#weighted rating of courses
C = df['popularity'].mean()
print("weighted rating of popularity (sectionCount): " + str(C)) # section count is popularity
# Calculate the minimum number of votes required to be in the chart, m
m = df['popularity'].quantile(0.92)
print("minimum number of rank required to be in the chart: " + str(m))
# Filter out all qualified classes into a new DataFrame
q_classes = df.copy().loc[df['popularity'] >= m]
q_classes.shape
#mimicing their catagorical classifier
def weighted_rating(x, m=m, C=C):
    v = x['popularity']
    R = x['popularity_avg']
    # Calculation based on the IMDB formula 
    return (v/(v+m) * R) + (m/(m+v) * C) # thing to modify for courses?
#adding new column with score
q_classes['score'] = q_classes.apply(weighted_rating, axis=1)
#sorting movvies based on ^ and printing
q_classes = q_classes.sort_values('score', ascending=False)
q_classes[['course_id', 'description', 'name', 'popularity', 'popularity_avg']].head(15)


weighted rating of popularity (sectionCount): 2.408450704225352
minimum number of rank required to be in the chart: 3.4000000000000057


,course_id,description,name,popularity,popularity_avg
2,CMSC132,Introduction to use of computers to solve prob...,Object-Oriented Programming II,23,9.549708
1,CMSC131,Introduction to programming and computer scien...,Object-Oriented Programming I,20,8.304094
10,CMSC330,The semantics of programming languages and the...,Organization of Programming Languages,20,8.304094
6,CMSC250,Fundamental mathematical concepts related to c...,Discrete Structures,18,7.473684
4,CMSC216,Machine representation of data including integ...,Introduction to Computer Systems,16,6.643275
11,CMSC351,A systematic study of the complexity of some e...,Algorithms,4,1.660819


In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#Content based Recommendations from Courses: 
df = pd.read_json(getCourseURL(), orient='columns')
df['description'].head()


#tfidf = TfidfVectorizer(stop_words) #stop words like in R!
tfidf = TfidfVectorizer(stop_words='english',max_features=500) #stop words like in R!
df['description'] = df['description'].fillna('') 

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


Enter the course code: gvpt
retrieving https://api.umd.io/v0/courses/list
https://api.umd.io/v0/courses/GVPT170,GVPT200,GVPT201,GVPT202,GVPT203,GVPT221,GVPT241,GVPT273,GVPT282,GVPT301,GVPT306,GVPT317,GVPT319C,GVPT319D,GVPT319E,GVPT319G,GVPT331,GVPT355,GVPT356,GVPT359F,GVPT388,GVPT388L,GVPT388M,GVPT388W,GVPT390,GVPT396,GVPT397,GVPT409A,GVPT409G,GVPT409H,GVPT409O,GVPT409W,GVPT420,GVPT423H,GVPT429E,GVPT432,GVPT439A,GVPT439D,GVPT439F,GVPT445,GVPT449G,GVPT449I,GVPT456,GVPT459E,GVPT459H,GVPT459M,GVPT459N,GVPT459P,GVPT459R,GVPT459V,GVPT461,GVPT474,GVPT475,GVPT477,GVPT722,GVPT743,GVPT771,GVPT799,GVPT803,GVPT808B,GVPT808C,GVPT828B,GVPT831,GVPT834,GVPT888P,GVPT898,GVPT899


(67, 500)

In [75]:
from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df['course_id']).drop_duplicates()
def get_recommendations(courseName, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[courseName]

    # Get the pairwsie similarity scores of all
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the courses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar classes
    sim_scores = sim_scores[1:6]

    # Get the classes indices
    courses_indices = [i[0] for i in sim_scores]

    # Return the top 6 most similar classes
    return df[['course_id', 'description']].iloc[courses_indices]

In [76]:
courseName = input(str('Enter course code and course level nummber: ')).upper().strip()
pd.set_option('display.max_colwidth', -1)
get_recommendations(courseName)

Enter course code and course level nummber: gvpt201


,course_id,description
23,GVPT396,"A required course for all honors students designed to emphasize library research, methodology, and writing skills in political science and political philosophy. A written proposal, bibliography and research design for an honors paper required of all students as a final project."
44,GVPT477,A study of the factors that influence individual vote choice and voter participation in the U.S. The course will introduce political science research pertaining to both topics and will engage current controversies over such things as political campaign laws and the various state and federal rules that govern election administration.
47,GVPT771,"This is the core seminar in American political behavior. The course wil deal with prominent theoretical and empirical issues in the areas of voting, public opinion, political participation and other aspects of political behavior in the American context."
22,GVPT390,"Introduction to game theory with applications to political science, economics and sociology. Topics include preference theory, expected utility theory, Nash equilibria, subgame perfection, repeated games, folk theorems, and evolutionary stability."
24,GVPT397,Individual reading and research. Preparation of an original paper.


In [130]:
import ipywidgets as widgets
from IPython.display import display
from __future__ import print_function
from ipywidgets import IntSlider, Output
from IPython.display import display
from IPython.display import clear_output

button = widgets.Button(description="Run")
clear = widgets.Button(description="Clear")

text = widgets.Text()
display(clear)

display(text)
display(button)


    
def on_button_clicked(b):
    print("Course: " + text.value)
    a = (get_recommendations(text.value))
    out = widgets.Output(layout={'border': '1px solid black'})
    display(a)
    
    
def clearalloutput(b):
    a.clear_output()

button.on_click(on_button_clicked)
clear.on_click(clearalloutput)






A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

Course: GVPT201


,course_id,description
23,GVPT396,"A required course for all honors students designed to emphasize library research, methodology, and writing skills in political science and political philosophy. A written proposal, bibliography and research design for an honors paper required of all students as a final project."
44,GVPT477,A study of the factors that influence individual vote choice and voter participation in the U.S. The course will introduce political science research pertaining to both topics and will engage current controversies over such things as political campaign laws and the various state and federal rules that govern election administration.
47,GVPT771,"This is the core seminar in American political behavior. The course wil deal with prominent theoretical and empirical issues in the areas of voting, public opinion, political participation and other aspects of political behavior in the American context."
22,GVPT390,"Introduction to game theory with applications to political science, economics and sociology. Topics include preference theory, expected utility theory, Nash equilibria, subgame perfection, repeated games, folk theorems, and evolutionary stability."
24,GVPT397,Individual reading and research. Preparation of an original paper.
